<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/spectrum_v2-1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Enviroment

## Loading Drive and folders

In [1]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [2]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Loading Necesary Libraries

In [3]:
# Importing necesary libraries
import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
import seaborn as sn
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

# Loading Data and Pre-Procesing

In [4]:
data_path=path+"/Spectrum"

qpsk_path="/QPSK/A.10hops100km/"
qam_path="/16QAM/A.10hops100km"

#file_name="/dataSet_Spectrum_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx"
file_name = "/QPSK_dummy.xlsx"
# Skipping first row since only contain "Frequency" headers
df_qpsk = pd.read_excel(data_path+qpsk_path+file_name, sheet_name = "Sheet1", skiprows=1)

#file_name="/dataSet_Spectrum_Samples_16QAM_75GHz_LongHaul_output_10x100.xlsx"
file_name = "/16QAM_dummy.xlsx"
df_qam = pd.read_excel(data_path+qam_path+file_name, sheet_name = "Sheet1", skiprows=1)

#Diff in column 5 to 1,286 0.03125

In [5]:
#dump_file = dict desireed to be saved
#filename = just the name of the file (file saved on "pickle_folder")

def pickle_out(dump_file, filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  pickleout = open(filepath, "wb")
  pickle.dump(dump_file, pickleout)
  pickleout.close()

def pickle_in(filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  picklein = open(filepath, "rb")
  loaded_dict = pickle.load(picklein)
  return loaded_dict

## Aggregator

In [6]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

In [7]:
def aggreagator_selection(stop_gap=25, total_num_of_data=1281, min_prime = False):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  # Select the prime number we would use to aggregation
  if min_prime == False:
    # Ask the user for the desired aggregator
    print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
    while True:
      user_selection = input('Prime number to conduct the aggregation? ')
      if int(user_selection) not in prime_nums:
        print('Please select one of the available prime numbers. (Current selection is not avaliable)')
      else:
        break
    return int(user_selection)
  else:
    user_selection = min(prime_nums)
    freq_selected = min(new_freq_offsets)
    print('Selected the least common denominator for aggregation ({}) with a frequency of {}.'.format(user_selection, freq_selected))
    return int(user_selection)

In [8]:
def aggreagator(df, stop_gap=25, total_num_of_data=1281, min_prime = False):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data, min_prime = min_prime)
  
  # Creating new df for aggregate values
  agg_df = pd.DataFrame()
  agg_df = df[df.columns[0:5]]

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)
    #print(init_column_cnt)
    # Get average of freq values for new header
    headers = list(new_df.columns.values)
    headers_array = np.asarray(headers)
    new_header_name = np.mean(headers_array)

    # Add aggragated values to new df
    agg_df[str(new_header_name)] = new_df.mean(axis = 1)
  agg_dict = agg_df.to_dict()
  return agg_df, agg_dict


In [22]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

def aggreagator_selection(stop_gap=25, total_num_of_data=1281, min_prime = False):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  if min_prime == False:
    # Ask the user for the desired aggregator
    print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
    while True:
      user_selection = input('Prime number to conduct the aggregation? ')
      if int(user_selection) not in prime_nums:
        print('Please select one of the available prime numbers. (Current selection is not avaliable)')
      else:
        break
    return int(user_selection)
  else:
    user_selection = min(prime_nums)
    freq_selected = min(new_freq_offsets)
    print('Selected the least common denominator for aggregation ({}) with a frequency of {}.'.format(user_selection, freq_selected))
    return int(user_selection)

def aggreagator(df, stop_gap=25, total_num_of_data=1281, min_prime = True):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data, min_prime = min_prime)
  
  # Creating new df of aggregate values
  agg_df = pd.DataFrame()
  agg_df = df[df.columns[0:5]]

  agg_column=0
  loop_cnt=0
  init_column_cnt = 5
  while agg_column<=total_num_of_data:

    # Obtain current last column to stop the range aggregation
    agg_column=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg_column]]

    # Get average of freq values for new header
    headers = list(new_df.columns.values)
    headers_array = np.asarray(headers)
    new_header_name = np.mean(headers_array)

    # Add aggragated values to new df
    agg_df[str(new_header_name)] = new_df.mean(axis = 1)

    init_column_cnt += (agg_num)
    loop_cnt+=1

  #Create dict of agg df
  agg_dict = agg_df.to_dict()
  return agg_df, agg_dict

In [32]:
# QPSK AGG
QPSK_agg_df, QPSK_agg_dict = aggreagator(df_qpsk, stop_gap=100, total_num_of_data=(df_qpsk.shape[1]-5), min_prime = True)
#pickle_out(QPSK_agg_dict, "test_agg_spectrum_QPSK_"+str(stop))

Selected the least common denominator for aggregation (3) with a frequency of 0.0625.


# Get model

In [63]:
def get_dummy_spectrum(df):
  dummy_df = df.groupby(['#node', 'power_dBm']).mean()
  dummy_df.drop(['sampleid', 'Channels'], axis = 1, inplace = True)
  dummy_dict = dummy_df.to_dict()
  
  return dummy_df, dummy_dict
  #Return a tuple of (node, power) as keys in the nested dict and indexes

In [52]:
# Dummy Model QPSK
QPSK_dummy_df, QPSK_dummy_dict = get_dummy_spectrum(df_qpsk)

#pickle_out(QPSK_dummy_dict, "dummy_AGG_spectrum_QPSK")

In [53]:
QPSK_dummy_df.head()

,,Distance_km,-20,-19.96875,-19.9375,-19.90625,-19.875,-19.84375,-19.8125,-19.78125,-19.75,...,19.71875,19.75,19.78125,19.8125,19.84375,19.875,19.90625,19.9375,19.96875,20
#node,power_dBm,,,,,,,,,,,,,,,,,,,,,
1,-2,0.0,-52.955056,-54.126681,-52.713840,-54.542630,-53.699036,-52.716651,-54.298754,-52.256947,-56.682823,...,-55.905776,-54.248709,-55.992318,-53.838975,-50.924380,-53.185248,-53.206559,-54.780695,-52.482275,-53.598605
2,-2,100.0,-51.813386,-53.194155,-52.697528,-52.418356,-52.486649,-50.289808,-54.422782,-54.110832,-55.149438,...,-53.005446,-55.507125,-54.494074,-50.651776,-54.789647,-53.624259,-51.423731,-55.126913,-57.266619,-51.729962
3,-2,200.0,-55.626777,-54.634329,-54.702958,-56.522545,-50.522361,-51.649987,-51.647787,-57.084303,-53.951639,...,-51.567208,-56.796925,-51.015703,-52.421725,-54.815529,-51.357553,-51.877358,-57.233009,-56.753557,-53.614796


# Get Data

In [60]:
# available filters are -2, -1, 0

def get_data_spectrum(df, node, power):
  #data_df = pd.DataFrame({'#node': [int(node)], 'power_dBm': [int(power)]})
  # Get selected node
  data_df = df[df['#node'].isin([int(node)])]

  # Get selected power
  data_df = data_df[data_df['power_dBm'].isin([int(power)])]

  data_dict = data_df.to_dict()
  
  return data_df, data_dict
  #Return a tuple of (node, power) as keys in the nested dict and indexes

In [61]:
data_df, data_dict = get_data_spectrum(df_qpsk, node=1, power=-2)

In [62]:
data_df.head()

,sampleid,Channels,Distance_km,power_dBm,#node,-20,-19.96875,-19.9375,-19.90625,-19.875,...,19.71875,19.75,19.78125,19.8125,19.84375,19.875,19.90625,19.9375,19.96875,20
0,1,1,0,-2,1,-50.396272,-51.195507,-52.325501,-57.685476,-51.730270,...,-50.701297,-50.425586,-57.242411,-54.978886,-58.984450,-52.289897,-50.007652,-56.562352,-50.661037,-57.857937
1,2,1,0,-2,1,-49.464198,-62.300068,-49.878752,-45.576386,-50.935177,...,-52.919979,-61.894104,-60.851791,-53.621297,-47.427855,-57.669031,-51.735427,-56.145979,-51.259631,-46.965564
2,3,1,0,-2,1,-52.100342,-54.825356,-49.933739,-53.418630,-55.604328,...,-52.437768,-48.572056,-58.180955,-50.003337,-49.935404,-52.854423,-52.839831,-56.938673,-48.511327,-53.268586
3,4,1,0,-2,1,-61.429210,-54.973444,-45.026184,-63.731247,-45.706566,...,-56.792207,-59.099399,-45.632244,-56.454966,-46.126678,-50.632960,-48.675022,-50.899182,-64.439508,-46.552580
4,5,1,0,-2,1,-51.777530,-57.344578,-55.337847,-52.396041,-53.875442,...,-48.008932,-51.073828,-49.675923,-57.762095,-49.534186,-59.334809,-59.237640,-62.395677,-46.607586,-69.244091


# Remove outliers